# Uczenie maszynowe 

Titanica Problem

Celem programu jest przedstawienie metod rozwiazywania problemu zwiazanego z uczeniem maszynowym.

W Titanic Problem mamy za zadanie stworzyć model, który potrafi w spósb jak najdokładniejszy określić kto przeżyje w czasie katastrofie statku.

Pobiranie danych z kaggle, a następnie odczyt danych.

In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train_data= pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

FileNotFoundError: ignored

In [ ]:
titanic_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
titanic_data.head()

In [ ]:
titanic_data.info()

In [ ]:
titanic_data.describe()

W tym momęcie brakuej 3 danych takich jak wiek, kabina, zaokrętowanie.

Dane jakie mamy to 5 danych kategori i 5 kolumn (imię i nazwisko, płeć, bilet, kabina, zaokrętowanie).

Dane liczbowe w których mamy 6 kolumn.

Posiadamy 3 określony typy danych liczbowych.


-Binary data 


-Continuous data 


-Categorical data encoded

In [ ]:
print('Survived:', titanic_data.Survived.unique())
print('Pclass:', titanic_data.Pclass.unique())
print('SibSp:', titanic_data.SibSp.unique())
print('Parch:', titanic_data.Parch.unique())

In [ ]:
print('Null Training data %:\n')

print(titanic_data.isnull().mean())

print('\nNull Test data data %:\n')
print(test_data.isnull().mean())

# W tym miejscu zaczynamy uzupełniać brakujące elementy. Uzuepłniamy dane potrzebne do treningu i do etapu testowania.

In [ ]:



embarked_mode = titanic_data['Embarked'].mode().values[0]
titanic_data['Embarked'].fillna(embarked_mode, inplace=True)


titanic_data['Cabin'].fillna('Missing', inplace=True) 


titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)


test_data['Cabin'].fillna('Missing', inplace=True) 


test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)


test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)
    

Filter accuracy: 0.9513513513513514
Filter accuracy: 0.9207920792079208
Filter accuracy: 0.848314606741573
Filter accuracy: 0.9617486338797814


# Rozpoczynamy ustalanie danych kategorii (Imię, płeć, bilet, kabina, zaokrętowany).


In [ ]:
print('Sex values:',titanic_data.Sex.unique(), test_data.Sex.unique())
print('Ticket values:',titanic_data.Ticket.unique().size,test_data.Ticket.unique().size)
print('Cabin values:',titanic_data.Cabin.unique().size,test_data.Cabin.unique().size)
print('Embarked values:',titanic_data.Embarked.unique(), test_data.Embarked.unique())

Train data accuracy: 0.8597081930415263
Saved


In [ ]:
temp = pd.get_dummies(titanic_data[['Sex']]) 
titanic_data = titanic_data.join(temp)

temp = pd.get_dummies(test_data[['Sex']]) 
test_data = test_data.join(temp)

temp = pd.get_dummies(titanic_data[['Pclass']].astype(str)) 
temp.head()
titanic_data = titanic_data.join(temp)

temp = pd.get_dummies(test_data[['Pclass']].astype(str)) 
test_data = test_data.join(temp)

temp = pd.get_dummies(titanic_data[['Embarked']]) 
titanic_data= titanic_data.join(temp)

temp = pd.get_dummies(test_data[['Embarked']]) 
test_data= test_data.join(temp)



from sklearn.preprocessing import LabelEncoder 

le = LabelEncoder() 
le.fit(titanic_data['Ticket']) 
titanic_data['Ticket'] = le.transform(titanic_data['Ticket']) 

le = LabelEncoder() 
le.fit(test_data['Ticket']) 
test_data['Ticket'] = le.transform(test_data['Ticket'])


le = LabelEncoder() 
le.fit(titanic_data['Cabin']) 
titanic_data['Cabin'] = le.transform(titanic_data['Cabin']) 

le = LabelEncoder() 
le.fit(test_data['Cabin']) 
test_data['Cabin'] = le.transform(test_data['Cabin'])

In [ ]:
titanic_data.columns

In [ ]:
titanic_data.head()

# Rozpoczynamy skalowanie funkcji. Wykreślamy histogramy. 

In [ ]:
sns.distplot(titanic_data['Age'])

In [ ]:
sns.distplot(titanic_data['Fare'])

In [ ]:
from sklearn.preprocessing import MaxAbsScaler 


data_unscaled = titanic_data[['Age','Fare']]

data_unscaled = data_unscaled.append(test_data[['Age','Fare']])

scaler = MaxAbsScaler() 
scaler.fit(data_unscaled) 
data_scaled = scaler.transform(data_unscaled)

titanic_data_scaled = data_scaled[:titanic_data.shape[0]]
test_data_scaled    = data_scaled[titanic_data.shape[0]+1:]

titanic_data_scaled = pd.DataFrame(titanic_data_scaled, columns = data_unscaled.columns) 
test_data_scaled = pd.DataFrame(test_data_scaled, columns = data_unscaled.columns) 


print(titanic_data_scaled.shape, test_data_scaled.shape)

# Sprawdźamy, czy dane zostały znormalizowane

In [ ]:
test_data_scaled.info

In [ ]:
titanic_data[['Age','Fare']] = titanic_data_scaled[['Age','Fare']]
test_data[['Age','Fare']] = test_data_scaled[['Age','Fare']]

# Sprawdźamy jakie zmiany zostały dokonane.

In [ ]:
sns.distplot(titanic_data['Age'])

In [ ]:
sns.distplot(titanic_data['Fare'])

# Rozpoczynamy test zbioru danych. Generujemy testowe zestawy danych.


In [ ]:
test_data.head()

In [ ]:
selected_x_columns = ['Age','SibSp','Parch', 'Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3' ]
train_x = titanic_data[selected_x_columns]
test_x =  test_data[selected_x_columns]

train_y = titanic_data['Survived']


In [ ]:
print(train_x.shape, test_x.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import train_test_split



model = Sequential()
model.add(Dense(30, input_dim=train_x.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(60, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
...

model.fit(train_x, train_y, epochs=300, batch_size=200)

In [ ]:
test_y = model.predict(test_x)

In [ ]:
test_y = np.where(test_y>0.8, 1,0)

test_y.shape

In [ ]:
result = test_data.join(pd.DataFrame(test_y, columns=['Survived']))
result = result[['PassengerId', 'Survived']]

In [ ]:
result

# Przesyłanie wyników.

In [ ]:
result.to_csv('submission.csv', index=False)

# Wnioski

Wynik jaki został wygnerowany są na poziome 0.8373 z maksymalną różnicą nie przekraczjącą dwóch setnych. W celu poprawy wyników należy rozważyć wiecej możliwych modeli. Otrzymane wyniki są w pełnii zadowalające.